In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-music-reviews/Musical_instruments_reviews.csv
/kaggle/input/amazon-music-reviews/Musical_Instruments_5.json


In [2]:
df = pd.read_csv('../input/amazon-music-reviews/Musical_instruments_reviews.csv')

In [3]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014"


In [4]:
df.overall.unique()

array([5., 3., 4., 2., 1.])

In [5]:
df.dropna(subset = ['reviewText'],inplace = True)

In [6]:
texts = df.reviewText + df.summary
labels = [1 if x > 3 else 0 for x in df.overall]

In [7]:
import re

In [8]:
import string

In [30]:
data = [sent.translate(str.maketrans('','',string.punctuation)) for sent in texts]

In [10]:
from nltk.corpus import stopwords

In [11]:
stops = stopwords.words('english')

In [22]:
from nltk.tokenize import word_tokenize

In [33]:
data = [word_tokenize(w.lower()) for w in data]

In [35]:
x = []
for  sent in data:
    x.append([w for w in sent if not w in stops])

In [37]:
from sklearn.feature_extraction.text import CountVectorizer

In [38]:
vec = CountVectorizer()

In [39]:
from sklearn.model_selection import train_test_split

In [48]:
dat = [' '.join(w) for w in x]

In [80]:
trainx,testx,trainy,testy = train_test_split(dat,labels)

In [54]:
vec.fit(trainx)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [55]:
trainx = vec.transform(trainx)
testx = vec.transform(testx)

In [56]:
from sklearn.naive_bayes import MultinomialNB

In [57]:
nb = MultinomialNB()

In [58]:
nb.fit(trainx,trainy)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [59]:
nb.score(testx,testy)

0.8868954758190327

In [60]:
from sklearn.ensemble import RandomForestClassifier

In [61]:
rf = RandomForestClassifier()

In [62]:
rf.fit(trainx,trainy)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [63]:
rf.score(testx,testy)

0.8857254290171607

In [65]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

In [91]:
trainx,testx,trainy,testy = train_test_split(dat,labels)

In [104]:
tok = Tokenizer()

In [93]:
import numpy as np
trainx = np.array(trainx)

In [107]:
tok.fit_on_texts(trainx)

In [109]:
trainx = tok.texts_to_sequences(trainx)
testx = tok.texts_to_sequences(testx)

In [113]:
trainy = to_categorical(trainy)
testy = to_categorical(testy)

In [115]:
from tensorflow.keras.layers import Embedding,Dense,Dropout,LSTM,GlobalAveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [116]:
trainx = pad_sequences(trainx,maxlen = 200,padding = 'post')
testx = pad_sequences(testx,maxlen = 200,padding = 'post')

In [130]:
model = Sequential([
    Embedding(len(tok.word_index)+1,50,input_length=200),
    LSTM(64,return_sequences=True),
    GlobalAveragePooling1D(),
    Dense(32,activation = 'relu'),
    Dropout(0.25),
    Dense(2,activation = 'softmax')
])

ValueError: Input 0 of layer global_average_pooling1d_1 is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: [None, 64]

In [131]:
model.compile('adam','binary_crossentropy',['accuracy'])

In [129]:
model.fit(trainx,trainy,validation_data=(testx,testy),epochs = 20,)

Train on 7690 samples, validate on 2564 samples
Epoch 1/20
7690/7690 [==============================] - 31s 4ms/sample - loss: 0.4045 - accuracy: 0.8736 - val_loss: 0.3411 - val_accuracy: 0.8924
Epoch 2/20
7690/7690 [==============================] - 29s 4ms/sample - loss: 0.3876 - accuracy: 0.8748 - val_loss: 0.3437 - val_accuracy: 0.8924
Epoch 3/20
5664/7690 [=====================>........] - ETA: 7s - loss: 0.3780 - accuracy: 0.8789

KeyboardInterrupt: 

In [123]:
model.evaluate(testx,testy)

2564/2564 [==============================] - 0s 46us/sample - loss: 0.8299 - accuracy: 0.8631


[0.8298928346165256, 0.8631045]